In [1]:
import tensorflow.keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization, LocallyConnected2D, SeparableConv2D
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.constraints import max_norm
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import activations

In [2]:
from data_collect import import_data

In [3]:
X, y_priors, y_values = import_data()

X_train, X_test, y_prior_train, y_prior_test, y_value_train, y_value_test = train_test_split(X, y_priors, y_values, test_size=0.2)

print(X_train.shape)
print(y_prior_train.shape)
print(y_value_train.shape)

(640000, 9, 9, 3)
(640000, 82)
(640000,)


In [4]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3, input_shape=X[0].shape, padding ="same"))
model.add(Conv2D(64, kernel_size=3, padding ="same"))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, padding ="same"))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(82, activation='sigmoid')) 

opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mae', optimizer=opt)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 64)          1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2

In [6]:
# hyperparamètres
epochs = 10
batch_size = 256

# training
print("Training ...")
model.fit(X_train, y_prior_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_prior_test))

Training ...
Epoch 1/10
2500/2500 [==============================] - 39s 16ms/step - loss: 0.0122 - val_loss: 0.0122
Epoch 2/10
2500/2500 [==============================] - 39s 15ms/step - loss: 0.0122 - val_loss: 0.0122
Epoch 3/10
2500/2500 [==============================] - 39s 15ms/step - loss: 0.0122 - val_loss: 0.0122
Epoch 4/10
 389/2500 [===>..........................] - ETA: 30s - loss: 0.0122

KeyboardInterrupt: 

In [7]:
proba = model.predict(X_test)
print(proba[6])
print(sum(proba[6]))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0


In [9]:
model.save('model/model_priors.h5')

In [10]:
model.pop()
model.add(Dense(1, activation='tanh'))
opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mae', optimizer=opt)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 64)          2368      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2

In [11]:
# hyperparamètres
epochs = 20
batch_size = 64

# training
print("Training ...")
model.fit(X_train, y_value_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_value_test))

Training ...
Epoch 1/20
606/606 [==============================] - 5s 7ms/step - loss: 0.0344 - val_loss: 0.0182
Epoch 2/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0198 - val_loss: 0.0138
Epoch 3/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0181 - val_loss: 0.0122
Epoch 4/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0169 - val_loss: 0.0115
Epoch 5/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0163 - val_loss: 0.0122
Epoch 6/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0158 - val_loss: 0.0176
Epoch 7/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0154 - val_loss: 0.0130
Epoch 8/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0154 - val_loss: 0.0151
Epoch 9/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0155 - val_loss: 0.0177
Epoch 10/20
606/606 [==============================] - 4s 7ms/step - loss: 0.0148 - val

In [12]:
proba = model.predict(X_test)
print(proba[:6].reshape((-1, 6)))
print(y_value_test[:6])

[[-0.42945614 -1.         -0.06603254 -0.3047543  -1.         -1.        ]]
[-0.42439024 -1.         -0.03       -0.31666667 -1.         -1.        ]


In [13]:
model.save('model/model_values.h5')